<a href="https://colab.research.google.com/github/WisdomerLee/Pytorch/blob/main/13Optimizer_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/pytorch/examples/tree/master/mnist

In [ ]:
import os
from glob import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import numpy as np

In [ ]:
seed = 1

batch_size = 64
test_batch_size = 64

no_cuda = False

In [ ]:
use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

Preprocess

In [ ]:
torch.manual_seed(seed)

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train = True, download = True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307, ), (0.3081,))
                   ])),
    batch_size = batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train = False, transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size = test_batch_size, shuffle = True)

Model

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1=nn.Conv2d(1,20,5,1)
        self.conv2=nn.Conv2d(20,50,5,1)
        self.fc1=nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x=F.max_pool2d(x,2,2)
        x=F.relu(self.conv2(x))
        x=F.max_pool2d(x,2,2)
        x=x.view(-1,4*4*50)
        x=F.relu(self.fc1(x))
        x=self.fc2(x)
        return F.log_softmax(x, dim=1)

Optimization
model, optimization 설정

In [ ]:
model = Net().to(device)

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5)

Parameters들 확인
Weight, Bais를 순서대로 보여줌

In [ ]:
params = list(model.parameters())
for i in range(8):
    print(params[i].size())

torch.Size([20, 1, 5, 5])
torch.Size([20])
torch.Size([50, 20, 5, 5])
torch.Size([50])
torch.Size([500, 800])
torch.Size([500])
torch.Size([10, 500])
torch.Size([10])


Before Training
학습하기 전에 model이 Train할 수 있도록 Train Mode로 변환
Convolution 또는 Linear 뿐만 아니라, Dropout과 추후에 배우게 될 Batch Normalization과 같이 parameter를 가진 Layer들도 학습하기 위해 준비

In [ ]:
model.train() #학습모드

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

모델에 넣기 위한 첫 Batch 데이터 추출

In [ ]:
data, target = next(iter(train_loader))

In [ ]:
data.shape, target.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

추출한 Batch 데이터를 cpu, gpu와 같은 device에 compile

In [ ]:
data, target = data.to(device), target.to(device)

In [ ]:
data.shape, target.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

gradient를 clear하여 새 최적화 값을 찾기 위한 준비

In [ ]:
optimizer.zero_grad()

준비한 데이터를 model에 input으로 넣어 output을 얻음

In [ ]:
output = model(data)

Model에서 예측한 결과를 Loss Function에 넣음
Negative Log-Likelihood Loss라는 Loss Function을 사용

In [ ]:
loss= F.nll_loss(output, target)

Back Propaganda로 Gradient 계산

In [ ]:
loss.backward()

계산된 Gradients: parameter에 Update

In [ ]:
optimizer.step()

Start Training
위의 최적화 과정을 반복하여 학습

In [ ]:
epochs =1
log_interval = 100

In [ ]:
for epoch in range(1, epochs+1):
    #Model 학습
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx% log_interval == 0:
            print('Training Epoch: {} [{}/{} ({:.0f}%)]\tLoss:{:6f}'.format(epoch, batch_idx*len(data), len(train_loader.dataset),
                                                                            100*batch_idx/len(train_loader), loss.item()))
    model.eval()

    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss +=F.nll_loss(output, target, reduction ='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
        
    test_loss /= len(test_loader.dataset)
    
    print('\nTest set: Average Loss : {:.4f}, Accuracy: {}/{}({:0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), 100.*correct/len(test_loader.dataset)))
    

Training Epoch: 1 [0/60000 (0%)]	Loss:0.643356
Training Epoch: 1 [6400/60000 (11%)]	Loss:0.454838
Training Epoch: 1 [12800/60000 (21%)]	Loss:0.403953
Training Epoch: 1 [19200/60000 (32%)]	Loss:0.480920
Training Epoch: 1 [25600/60000 (43%)]	Loss:0.380319
Training Epoch: 1 [32000/60000 (53%)]	Loss:0.338364
Training Epoch: 1 [38400/60000 (64%)]	Loss:0.281253
Training Epoch: 1 [44800/60000 (75%)]	Loss:0.383205
Training Epoch: 1 [51200/60000 (85%)]	Loss:0.335981
Training Epoch: 1 [57600/60000 (96%)]	Loss:0.375978

Test set: Average Loss : 0.2994, Accuracy: 9092/10000(90.920000%)



Evaluation
model.train()모드로 변한 것처럼 평가할 때는 model.eval()로 설정
batch normalization, dropout같은 layer들을 잠금

In [ ]:
model.eval()

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

autograd engine : backpropagation이나 gradient계산을 끔 : memory usage를 줄여 속도를 높임

In [ ]:
test_loss=0
correct=0

with torch.no_grad():
    data, target = next(iter(test_loader))
    data, target = data.to(device), target.to(device)
    output = model(data)
    
    test_loss+= F.nll_loss(output, target, reduction = 'sum').item() #이렇게 sum으로 reduction을 해주면 계산한 결과를 하나로 합쳐 
    
    pred = output.argmax(dim=1, keepdim=True)
    correct = pred.eq(target.view_as(pred)).sum().item()

In [ ]:
test_loss

33.741512298583984

In [ ]:
correct

53

In [ ]:
pred.shape

torch.Size([64, 1])

In [ ]:
output.shape

torch.Size([64, 10])

In [ ]:
target.view_as(pred).shape

torch.Size([64, 1])

In [ ]:
pred.eq(target.view_as(pred)).sum().item()/64

0.828125

In [ ]:
test_loss

33.741512298583984

In [ ]:
correct

53

In [ ]:
test_loss /= len(test_loader.dataset)

In [ ]:
test_loss

0.0033741512298583985

In [ ]:
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss +=F.nll_loss(output, target, reduction ='sum').item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        
test_loss /= len(test_loader.dataset)

print('\nTest set: Average Loss : {:.4f}, Accuracy: {}/{}({:0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset), 100.*correct/len(test_loader.dataset)))


Test set: Average Loss : 0.4799, Accuracy: 8662/10000(86.620000%)

